In [3]:
import os
import json
from pymongo import MongoClient

In [23]:
# Чтение данных из JSON файла
with open("books_from_books.toscrape.com.json", "r", encoding='utf-8') as file:
    data = json.load(file)

In [24]:
# Соединение с MongoDB
client = MongoClient('localhost', 27017)
db = client['my_books_database']

In [25]:
# Создание коллекции на основании данных из JSON файла
collection_name = "Список книг от 12-03-2024"
for item in data:
    collection = db[collection_name]
    collection.insert_one(item)

client.close()

In [26]:
# Установление связи с MongoDB
client = MongoClient('localhost', 27017)
db = client['my_books_database']
collection = db['Список книг от 12-03-2024']

In [27]:
#### Просмотр количества документов в коллекции
document_count = collection.count_documents({})
print("Количества документов в коллекции:", document_count)

Количества документов в коллекции: 1000


In [28]:
# Запрос количества и названий книг, дороже 50 фунтов
query = {"Цена в фунтах стерлингов": {"$gt": 59.0}}

# Получения документов, соответствующих запросу
documents = collection.find(query)

print("Количество книг дороже 59 фунтов - ", collection.count_documents(query), ":")

# Отображение названий книг
for document in documents:
    print("Название книги:", document['Название'])

Количество книг дороже 59 фунтов -  12 :
Название книги: Thomas Jefferson and the Tripoli Pirates: The Forgotten War That Changed American History
Название книги: The Gray Rhino: How to Recognize and Act on the Obvious Dangers We Ignore
Название книги: The Diary of a Young Girl
Название книги: Boar Island (Anna Pigeon #19)
Название книги: The Improbability of Love
Название книги: The Man Who Mistook His Wife for a Hat and Other Clinical Tales
Название книги: The Barefoot Contessa Cookbook
Название книги: Last One Home (New Beginnings #1)
Название книги: The Perfect Play (Play by Play #1)
Название книги: The Bone Hunters (Lexy Vaughan & Steven Macaulay #2)
Название книги: Life Without a Recipe
Название книги: Civilization and Its Discontents


In [29]:
### Запрос для определения максимальной и минимальной цены книги в коллекции

# Определение конвеера для агрегации данных
pipeline = [
    {"$group": {"_id": None, "max_price": {"$max": "$Цена в фунтах стерлингов"}, "min_price": {"$min": "$Цена в фунтах стерлингов"}}}
]

# Применение конвеера
result = list(collection.aggregate(pipeline))

# Получение минимального и максимального значений
max_price = result[0]["max_price"]
min_price = result[0]["min_price"]

# Печать результатов
print("Минимальная цена:", min_price)
print("Максимальная цена:", max_price)

Минимальная цена: 10.0
Максимальная цена: 59.99


In [31]:
# Создание конвеера для разбиения всех книг по стоимости на три категории: низкая стоимость (дешевле 15 фунтов), средня стоимость (от 15 до 25 фунтов) и высокая стоимость (выше 25 фунтов). Затем подсчет количества книг в каждой категории агрегацией по полю "Количество в наличии"
pipeline_by_group = [
    {
        "$group": {
            "_id": {
                "$cond": {
                    "if": {"$lte": ["$Цена в фунтах стерлингов", 15]},
                    "then": "Низкая стоимость",
                    "else": {
                        "$cond": {
                            "if": {"$lte": ["$Цена в фунтах стерлингов", 25]},
                            "then": "Средная стоимость",
                            "else": "Высокая стоимость"
                        }
                    }
                }
            },
            "Количество в наличии": {"$sum": "$Количество в наличии"}
        }
    }
]

# Выполнение конвеера
result_by_group = list(collection.aggregate(pipeline_by_group))

# Печать результатов
for category in result_by_group:
    print("Категория по стоимости:", category["_id"])
    print("Количество в наличии:", category["Количество в наличии"])
    print()

# Создание конвеера для определения суммарного количества всех книг по полю "Количество в наличии"
pipeline_total_count = [
    {
        "$group": {
            "_id": None,
            "total_count": {"$sum": "$Количество в наличии"}
        }
    }
]

# Выполнение конвеера
result_total_count = list(collection.aggregate(pipeline_total_count))

# Отображение результата
total_count = result_total_count[0]['total_count'] if result_total_count else 0
print("Общее количество книг в наличии:", total_count) #### совпадает с суммой книг в наличии по категориям стоимости

Категория по стоимости: Низкая стоимость
Количество в наличии: 912

Категория по стоимости: Средная стоимость
Количество в наличии: 1722

Категория по стоимости: Высокая стоимость
Количество в наличии: 5951

Общее количество книг в наличии: 8585


Зарегистрируйтесь в ClickHouse.
Загрузите данные в ClickHouse и создайте таблицу для их хранения

In [1]:
!pip install clickhouse_driver

   ---------------------------------------- 0.0/212.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/212.2 kB ? eta -:--:--
   ----- --------------------------------- 30.7/212.2 kB 262.6 kB/s eta 0:00:01
   ------- ------------------------------- 41.0/212.2 kB 245.8 kB/s eta 0:00:01
   ------------- ------------------------- 71.7/212.2 kB 357.2 kB/s eta 0:00:01
   --------------------------- ---------- 153.6/212.2 kB 573.4 kB/s eta 0:00:01
   ------------------------------------ - 204.8/212.2 kB 731.4 kB/s eta 0:00:01
   -------------------------------------- 212.2/212.2 kB 645.3 kB/s eta 0:00:00


In [29]:
from clickhouse_driver import Client

In [32]:
client = Client(host = "b3xdjlz1rs.europe-west4.gcp.clickhouse.cloud",
                user = 'default',
                secure = True,
                port = 9440, 
                password = '8Z0vK_Stx~MLg')


Вопрос к преподователю подключен к порту с лекции 9440 мой порт 8443 выбивает ошибку с чем связано? подключение к клик хаус осущественно с помощю впн. в этом причина?

In [28]:
client.execute('SHOW TABLES')

[('trips',)]

In [33]:
client.execute (
'''
CREATE TABLE trips
(
    `trip_id` UInt32,
    `vendor_id` Enum8('1' = 1, '2' = 2, '3' = 3, '4' = 4, 'CMT' = 5, 'VTS' = 6, 'DDS' = 7, 'B02512' = 10, 'B02598' = 11, 'B02617' = 12, 'B02682' = 13, 'B02764' = 14, '' = 15),
    `pickup_date` Date,
    `pickup_datetime` DateTime,
    `dropoff_date` Date,
    `dropoff_datetime` DateTime,
    `store_and_fwd_flag` UInt8,
    `rate_code_id` UInt8,
    `pickup_longitude` Float64,
    `pickup_latitude` Float64,
    `dropoff_longitude` Float64,
    `dropoff_latitude` Float64,
    `passenger_count` UInt8,
    `trip_distance` Float64,
    `fare_amount` Float32,
    `extra` Float32,
    `mta_tax` Float32,
    `tip_amount` Float32,
    `tolls_amount` Float32,
    `ehail_fee` Float32,
    `improvement_surcharge` Float32,
    `total_amount` Float32,
    `payment_type` Enum8('UNK' = 0, 'CSH' = 1, 'CRE' = 2, 'NOC' = 3, 'DIS' = 4),
    `trip_type` UInt8,
    `pickup` FixedString(25),
    `dropoff` FixedString(25),
    `cab_type` Enum8('yellow' = 1, 'green' = 2, 'uber' = 3),
    `pickup_nyct2010_gid` Int8,
    `pickup_ctlabel` Float32,
    `pickup_borocode` Int8,
    `pickup_ct2010` String,
    `pickup_boroct2010` String,
    `pickup_cdeligibil` String,
    `pickup_ntacode` FixedString(4),
    `pickup_ntaname` String,
    `pickup_puma` UInt16,
    `dropoff_nyct2010_gid` UInt8,
    `dropoff_ctlabel` Float32,
    `dropoff_borocode` UInt8,
    `dropoff_ct2010` String,
    `dropoff_boroct2010` String,
    `dropoff_cdeligibil` String,
    `dropoff_ntacode` FixedString(4),
    `dropoff_ntaname` String,
    `dropoff_puma` UInt16
)
ENGINE = MergeTree
PARTITION BY toYYYYMM(pickup_date)
ORDER BY pickup_datetime;
'''
)

ServerException: Code: 57.
DB::Exception: Table default.trips already exists. Stack trace:

0. DB::Exception::Exception(DB::Exception::MessageMasked&&, int, bool) @ 0x000000000cab03fb in /usr/bin/clickhouse
1. DB::Exception::Exception<String, String>(int, FormatStringHelperImpl<std::type_identity<String>::type, std::type_identity<String>::type>, String&&, String&&) @ 0x000000000739181d in /usr/bin/clickhouse
2. DB::InterpreterCreateQuery::doCreateTable(DB::ASTCreateQuery&, DB::InterpreterCreateQuery::TableProperties const&, std::unique_ptr<DB::DDLGuard, std::default_delete<DB::DDLGuard>>&) @ 0x000000001184def4 in /usr/bin/clickhouse
3. DB::InterpreterCreateQuery::createTable(DB::ASTCreateQuery&) @ 0x000000001183c311 in /usr/bin/clickhouse
4. DB::InterpreterCreateQuery::execute() @ 0x0000000011824d0f in /usr/bin/clickhouse
5. DB::executeQueryImpl(char const*, char const*, std::shared_ptr<DB::Context>, DB::QueryFlags, DB::QueryProcessingStage::Enum, DB::ReadBuffer*) @ 0x0000000011dd6154 

In [23]:
client.execute(
'''
INSERT INTO trips
SELECT * FROM s3(
    'https://datasets-documentation.s3.eu-west-3.amazonaws.com/nyc-taxi/trips_{1..2}.gz',
    'TabSeparatedWithNames', "
    `trip_id` UInt32,
    `vendor_id` Enum8('1' = 1, '2' = 2, '3' = 3, '4' = 4, 'CMT' = 5, 'VTS' = 6, 'DDS' = 7, 'B02512' = 10, 'B02598' = 11, 'B02617' = 12, 'B02682' = 13, 'B02764' = 14, '' = 15),
    `pickup_date` Date,
    `pickup_datetime` DateTime,
    `dropoff_date` Date,
    `dropoff_datetime` DateTime,
    `store_and_fwd_flag` UInt8,
    `rate_code_id` UInt8,
    `pickup_longitude` Float64,
    `pickup_latitude` Float64,
    `dropoff_longitude` Float64,
    `dropoff_latitude` Float64,
    `passenger_count` UInt8,
    `trip_distance` Float64,
    `fare_amount` Float32,
    `extra` Float32,
    `mta_tax` Float32,
    `tip_amount` Float32,
    `tolls_amount` Float32,
    `ehail_fee` Float32,
    `improvement_surcharge` Float32,
    `total_amount` Float32,
    `payment_type` Enum8('UNK' = 0, 'CSH' = 1, 'CRE' = 2, 'NOC' = 3, 'DIS' = 4),
    `trip_type` UInt8,
    `pickup` FixedString(25),
    `dropoff` FixedString(25),
    `cab_type` Enum8('yellow' = 1, 'green' = 2, 'uber' = 3),
    `pickup_nyct2010_gid` Int8,
    `pickup_ctlabel` Float32,
    `pickup_borocode` Int8,
    `pickup_ct2010` String,
    `pickup_boroct2010` String,
    `pickup_cdeligibil` String,
    `pickup_ntacode` FixedString(4),
    `pickup_ntaname` String,
    `pickup_puma` UInt16,
    `dropoff_nyct2010_gid` UInt8,
    `dropoff_ctlabel` Float32,
    `dropoff_borocode` UInt8,
    `dropoff_ct2010` String,
    `dropoff_boroct2010` String,
    `dropoff_cdeligibil` String,
    `dropoff_ntacode` FixedString(4),
    `dropoff_ntaname` String,
    `dropoff_puma` UInt16
") SETTINGS input_format_try_infer_datetimes = 0
'''
)

Error on b3xdjlz1rs.europe-west4.gcp.clickhouse.cloud:9440 ping: Unexpected EOF while reading bytes
Connection was closed, reconnecting.


[]

In [34]:
client.execute("SELECT count() FROM trips")

[(1999657,)]

In [35]:
tip_amount = client.execute("SELECT round(avg(tip_amount), 2) FROM trips")  
print(tip_amount)

[(1.68,)]


In [36]:
type(tip_amount)

list

In [37]:
tip_amount[0][0]

1.68

In [38]:
avg_price = client.execute(
'''
SELECT
    passenger_count,
    ceil(avg(total_amount),2) AS average_total_amount
FROM trips
GROUP BY passenger_count
'''
)

Error on b3xdjlz1rs.europe-west4.gcp.clickhouse.cloud:9440 ping: Unexpected EOF while reading bytes
Connection was closed, reconnecting.


In [39]:
import pandas as pd

In [40]:
df = pd.DataFrame(avg_price, columns=("passenger_count", "average_total_amount"))

In [41]:
df

,passenger_count,average_total_amount
0,0,22.69
1,1,15.97
2,2,17.15
3,3,16.76
4,4,17.33
5,5,16.35
6,6,16.04
7,7,59.80
8,8,36.41
9,9,9.81
